In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

In [17]:
df = pd.read_parquet(r"data/df_kpi_perf_2108_0110.parquet", engine="pyarrow")
config = pd.read_parquet(r"data/df_huawei_config_2108_0110.parquet", engine="pyarrow")

In [18]:
#tirando as horas do df pra poder dar match
df['Data']=df['Data'].dt.date
df['Data']

0          2023-08-21
1          2023-08-21
2          2023-08-21
3          2023-08-21
4          2023-08-21
              ...    
1567446    2023-10-01
1567447    2023-10-01
1567448    2023-10-01
1567449    2023-10-01
1567450    2023-10-01
Name: Data, Length: 1567451, dtype: object

In [24]:
config['DLEARFCN'].unique()

array([  100,  1542,  2800,  3375,  9410,  9510,  3200,  1550,  3150,
        3350,    75,  3100,  3075,   525,  1799, 39250, 39450,  1300,
         547,  9362,  1470,   175])

In [ ]:
pivot_df = (
    config.pivot_table(
        index=[
            "LOCALCELLID",
            "dateday",
            "cellreselpriority_x",
            "qrxlevmin_x",
            "snonintrasearch",
            "thrshservlow",
            "cellreselpriority_y",
            "qrxlevmin_y",
            "threshxhigh",
            "threshxlow"
        ],
        columns="DLEARFCN",
        values="qoffsetfreq",
        aggfunc="first"  
    )
    .reset_index()
    .fillna(0)  
)

pivot_df.columns.name = None
pivot_df.head()


,LOCALCELLID,dateday,cellreselpriority_x,qrxlevmin_x,snonintrasearch,thrshservlow,cellreselpriority_y,qrxlevmin_y,threshxhigh,threshxlow,...,3100,3150,3200,3350,3375,9362,9410,9510,39250,39450
0,1,2023-09-18,6,-60,9,1,5,-58.0,1,0,...,0,0,0,0,0,0,dB18,0,0,0
1,1,2023-09-18,6,-60,9,1,7,-58.0,1,0,...,dB0,0,0,0,0,0,0,0,0,0
2,1,2023-09-18,6,-60,15,1,5,-64.0,11,11,...,0,0,dB0,0,0,0,0,0,dB0,dB0
3,1,2023-09-18,6,-60,15,1,5,-58.0,1,0,...,0,0,0,0,0,0,dB20,0,dB0,dB0
4,1,2023-09-18,6,-60,15,1,7,-58.0,1,0,...,dB0,dB-10,dB-20,0,0,0,0,0,0,0


In [ ]:
#falta juntar agora as duas tabelas, tem que linkar pela eNodeB e Date como chave primaria
#arrumar alguma forma melhor de lidar com os valores NAs, não é uma boa pratica botar tudo 0 nesse caso
#se livrar desses dbs antes dos numeros das frequencias... acho que faz sentido